In [2]:
# Importing Libraries
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import numpy as np
import pandarallel
from sklearn.preprocessing import StandardScaler, Normalizer, Binarizer
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from kerastuner.tuners import RandomSearch, Sklearn
from sklearn.model_selection import train_test_split
from kerastuner.engine.hyperparameters import HyperParameters
from keras.activations import relu
from keras.optimizers import RMSprop
import math

KeyboardInterrupt: 

In [ ]:
filename = "./EDA Notebook.ipynb.csv"

In [ ]:
df = pd.read_csv(filename)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Avoiding data leakage by dropping this here. Need to add this back into the EDA notebook and delete this
df.drop(columns='log_price', inplace=True)

In [ ]:
# Have to reset the index after importing CSV from EDA notebook. Indexes just don't carryover when 
# converted to output data
df.set_index("id", inplace=True)

In [ ]:
# Splitting data into target and feature matrix
target = 'price'
X = df.drop(columns=target)
y = df[target]

In [ ]:
# Sanity Check
assert len(X) == len(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Building Our Model
---
- Without preprocessing this time around

In [ ]:
def build_regression_model(hp):
    
    """
    Returns a complied keras model ready for keras-tuner gridsearch algorithms 
    """
    
    model = Sequential()
    
    # hidden layer
    model.add(Dense(units=hp.get('units'),activation=hp.get("activation")))
    
    # output layer
    model.add(Dense(1, activation='relu'))
    
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(hp.get('learning_rate')),
        loss='mse',
        metrics=['mae'])
    
    return model

In [ ]:
# build out our hyperparameter dictionary 
hp = HyperParameters()
hp.Int('units', min_value=32, max_value=512, step=32)
hp.Choice('learning_rate',values=[1e-1, 1e-2, 1e-3])
hp.Choice('activation',values=["relu"])

In [ ]:
X

In [ ]:
n_unique_hparam_combos = len(range(32,512+32, 32)) * 3 *2
n_param_combos_to_sample = n_unique_hparam_combos * .25

In [ ]:
random_tuner = RandomSearch(
            build_regression_model,
            objective='val_accuracy',
            max_trials=n_param_combos_to_sample, # number of times to sample the parameter set and build a model 
            seed=1234,
            hyperparameters=hp, # pass in our hyperparameter dictionary
            directory='./keras-tuner-trial',
            project_name='random_search')

In [ ]:
# take note of Total elapsed time in print out
random_tuner.search(X_train,
                    y_train,
                    epochs=3,
                    validation_data=(X_test, y_test))
# Need to go back to EDA Notbeook and finish preprocessing data before I can do this
# Can use this as reference: https://stackoverflow.com/questions/58636087/tensorflow-valueerror-failed-to-convert-a-numpy-array-to-a-tensor-unsupporte